In [8]:
from pathlib import Path # type: ignore
from freyja_plot import FreyjaPlotter
import pandas as pd
import sys
import scipy.stats as stats
from collections import defaultdict
import plotly.io as pio
if (module_path:=str(Path(".").absolute().resolve().parent)) not in sys.path:
    sys.path.insert(0, module_path)
from sample_info import get_stats, summary_dict, expected, artic_runs as runs, renameSamples, plotting_dir, p_value_table

In [2]:
outdir = plotting_dir / "percent_of_expected/observed_vs_expected"
outdir.mkdir(exist_ok=True)

## ANOVA comapring percent of expected across tools (just MB)

In [3]:
file_map = {expected:"Expected"}
for tool, files in runs.items():
    for name,file in files.items():
        if "WB" == name:
        # if "NWRB" == name:
        # if "PWRB" == name:
            file_map[file] = tool
plotter = FreyjaPlotter(
    file_map=file_map,
    # colormap=colormap,
    summary_dict=summary_dict,
)
plotter = renameSamples(plotter)
df = plotter.plotPercentExpectedBox(summarized=True,return_df=True)
df

,Sample name,lineages,abundances,scheme,expected_abundance,percent_of_expected
115,0adgio1,Alpha,0.262,Alcov,0.250000,104.8
289,0adgio1-2,Alpha,0.145,Alcov,0.142857,101.5
106,0adgio1o2o3o4o5,Alpha,0.065,Alcov,0.066667,97.5
137,0agio1o2,Alpha,0.068,Alcov,0.062500,108.8
300,0aio1o2o3o4o5,Alpha,0.134,Alcov,0.125000,107.2
...,...,...,...,...,...,...
1832,0adgio1-2,Wuhan-hu-1,0.508,kallisto (C-WAP),0.285714,177.8
1384,0adgio1o2o3o4o5,Wuhan-hu-1,0.207,kallisto (C-WAP),0.133333,155.25
1427,0agio1o2,Wuhan-hu-1,0.256,kallisto (C-WAP),0.125000,204.8
1847,0aio1o2o3o4o5,Wuhan-hu-1,0.231,kallisto (C-WAP),0.125000,184.8


In [9]:
tukey_df = get_stats(df, value_col="percent_of_expected", p_min=0.01, batch_col="scheme")
p_table = p_value_table(tukey_df)
p_table

ANOVA for None samples comparing percent of expected across batches
p-value: 2.2941383910788995e-12	f-value: 9.438223956048526
The percent of expected was significantly different across Alcov (mean=86.56931034482756, std. dev.=30.07212149136993), Freyja (mean=86.3622304340426, std. dev.=37.23119280846171), kallisto (mean=59.96308333333334, std. dev.=26.850927649171688), kallisto (C-WAP) (mean=92.01730158730162, std. dev.=48.26184011312391), LCS (mean=78.21779215686274, std. dev.=36.9248852257005), lineagespot (mean=56.882857142857134, std. dev.=61.489998567324136), LolliPop (mean=96.6667076923077, std. dev.=44.75210184850989), VaQuERo (mean=87.65988477366258, std. dev.=29.368641783379186), and Kraken 2 (C-WAP) (mean=127.79676190476188, std. dev.=114.53345473528887), as determined by one-way ANOVA (F=9.438223956048526, p=2.2941383910788995e-12<0.01).
No significant difference in percent of expected (p-value=1.0>0.01) was found between Alcov and Freyja.

Tukey's HSD results:


In [11]:
pio.write_image(p_table, outdir / "p_value_table.jpg", width=1200, height=550)

## ANOVA and Tukey's HSD results when comparing O/E for each background for each tool

In [2]:
p_min = 0.01
tools = runs.keys()
treatments = ["WB","NWRB","PWRB"]
report = defaultdict(list)
any_significant_p = False
for tool, files in runs.items():
    # print(tool)

    dfs = []
    for treatment in treatments:
        file_map = {expected:"Expected"}
        for background,file in files.items():
            if treatment == background:
                file_map[file] = tool
        # print(file_map)
        plotter = FreyjaPlotter(
            file_map=file_map,
            # colormap=colormap,
            summary_dict=summary_dict,
        )
        plotter = renameSamples(plotter)
        df = plotter.plotPercentExpectedBox(summarized=True,return_df=True)
        df["batch"] = treatment
        dfs.append(df)
    anova_df = pd.concat(dfs)
    # print(anova_df)
    p_value, statistic, mean_info, tukey_df = get_stats(anova_df, value_col="percent_of_expected", p_min=p_min, return_tuple=True)
    if tukey_df is not None:
        if p_value < p_min:
            any_significant_p = True
        report[tool] = f"p-value={p_value}, F={statistic}"
        report[tool] = tukey_df
        # print(tukey_df)
if any_significant_p:
    for tool, info in report.items():
        print(tool)
        for line in info:
            print(line)
        print("\n\n")
else:
    print(f"No significant results with a p-value < {p_min}.")


No significant results with a p-value < 0.01.


## ANOVA and Tukey's HSD results when comparing O/E for each background for each lineage

In [3]:
p_min = 0.01
lineages = ["Wuhan-hu-1","Alpha","Gamma","Delta","Iota","BA.1.X","BA.2.X","BG.X","BA.4.X","BA.5.X"]
# treatments = {"Control":"WB","Neg Spike-in":"NWRB","Pos Spike-in":"PWRB"}
treatments = ["WB","NWRB","PWRB"]
treatment_pairs = [["WB","NWRB"],["WB","PWRB"],["NWRB","PWRB"]]
anova_results = {"Lineage":[],"F-statistic":[],"p-value":[]}
tukey_results = defaultdict(list)
for lineage in lineages:
    print(lineage)
    anova_results["Lineage"].append(lineage)
    tukey_results["Lineage"].append(lineage)

    percents = {}
    dfs = []
    for treatment in treatments:
        file_map = {expected:"Expected"}
        for tool, files in runs.items():
            for name,file in files.items():
                if treatment == name:
                    file_map[file] = tool
        # print(file_map)
        plotter = FreyjaPlotter(
            file_map=file_map,
            # colormap=colormap,
            summary_dict=summary_dict,
        )
        plotter = renameSamples(plotter)
        df = plotter.plotPercentExpectedBox(summarized=True,return_df=True)
        percents[treatment] = list(df.loc[df["lineages"]==lineage, "percent_of_expected"])
        df = df[df["lineages"]==lineage]
        df["batch"] = treatment
        dfs.append(df)
    anova_df = pd.concat(dfs)
    p_value, statistic, mean_info, tukey_df = get_stats(anova_df, value_col="percent_of_expected", p_min=p_min, return_tuple=True)
    anova_results["F-statistic"].append(statistic)
    anova_results["p-value"].append(p_value)

    if tukey_df is not None:
        print(tukey_df)
        for treatment_pair in treatment_pairs:
            pair_name = " vs ".join(treatment_pair)
            tukey_results[pair_name].append(tukey_df.loc[treatment_pair[0],treatment_pair[1]])
    else:
        for treatment_pair in treatment_pairs:
            pair_name = " vs ".join(treatment_pair)
            tukey_results[pair_name].append("-")
    print("\n\n")
anova = pd.DataFrame(anova_results)
tukey = pd.DataFrame(tukey_results)

Wuhan-hu-1



Alpha
4
[1.7909560765796443e-06, 14.204049898015432, 'WB (mean=90.0815007020202, std. dev.=26.296234414874373), NWRB (mean=72.21485155913977, std. dev.=21.20314535929819), and PWRB (mean=71.82217631249999, std. dev.=18.56435137773008), as determined by one-way ANOVA (F=14.204049898015432, p=1.7909560765796443e-06<0.01).',         WB   NWRB   PWRB
WB    <NA>    0.0    0.0
NWRB   0.0   <NA>  0.995
PWRB   0.0  0.995   <NA>]
        WB   NWRB   PWRB
WB    <NA>    0.0    0.0
NWRB   0.0   <NA>  0.995
PWRB   0.0  0.995   <NA>



Gamma
4
[0.00010274766957374775, 9.73827666366686, 'WB (mean=75.7026555493827, std. dev.=15.496448058787792), NWRB (mean=63.51943741509434, std. dev.=19.81286819679811), and PWRB (mean=60.58794284567901, std. dev.=20.863743930494056), as determined by one-way ANOVA (F=9.73827666366686, p=0.00010274766957374775<0.01).',          WB   NWRB   PWRB
WB     <NA>  0.003    0.0
NWRB  0.003   <NA>  0.701
PWRB    0.0  0.701   <NA>]
         WB   NWRB   PWRB
WB    

In [13]:
# data for table 2 in paper
table = anova.merge(tukey, on="Lineage")
table.to_csv(outdir/"table2_data_lineage_anova_each_bg.csv")
table

,Lineage,F-statistic,p-value,WB vs NWRB,WB vs PWRB,NWRB vs PWRB
0,Wuhan-hu-1,0.047882,9.532785e-01,-,-,-
1,Alpha,14.204050,1.790956e-06,0.0,0.0,0.995
2,Gamma,9.738277,1.027477e-04,0.003,0.0,0.701
3,Delta,2.996111,5.347146e-02,-,-,-
4,Iota,23.531721,1.229030e-09,0.003,0.0,0.002
5,BA.1.X,0.025037,9.752771e-01,-,-,-
6,BA.2.X,1.281574,2.793544e-01,-,-,-
7,BG.X,2.418755,9.176943e-02,-,-,-
8,BA.4.X,2.470374,8.652837e-02,-,-,-
9,BA.5.X,13.120129,3.736958e-06,0.044,0.0,0.02
